In [1]:
# Permet de jouer au jeu
import retro
# Permet de ralentir la vitesse 
import time

## Preprocess

In [2]:
from gym import Env
from gym.spaces import MultiBinary, Box
import numpy as np
import cv2
import matplotlib.pyplot as plt

In [3]:
# Environnement Customisé
class StreetFighter(Env) : 
    def __init__(self) :
        super().__init__()
        # A fixer inférieure à 200
        self.taille_reduite = 84
        # Specification de l'espace des actions et de l'espace d'observation
        # low = 0, high = 255 : couleur pixels par défaut
        # shape : shape de la sortie par défaut (hauteur, largeur, Gris)
        self.observation_space = Box(low=0,high=255,shape=(self.taille_reduite,self.taille_reduite,1),dtype=np.uint8)
        # action_space = MultiBinary(12) : 12 touches possibles et combinables pour faire des coups spéciaux
        self.action_space = MultiBinary(12)
        # Lancer une instance du jeu et ne permet que les combinaisons valides de boutons
        self.game = retro.make(game='StreetFighterIISpecialChampionEdition-Genesis', use_restricted_actions = retro.Actions.FILTERED)
    
    def step(self,action):
        # Faire une étape 
        observation, reward, done, info = self.game.step(action)
        observation = self.preprocess(observation)
        
        # Fonction de récompense
        reward = info['score'] - self.score
        self.score = info['score']
        
        return observation, reward, done, info
    
    def render(self,*args,**kwargs):
        self.game.render()
    
    def reset(self):
        # Remet le jeu à zéro
        observation = self.game.reset()
        # Preprocess l'image obtenue
        observation = self.preprocess(observation)
        # Cette variable va permettre de stocker la récompense obtenue pour la partie
        self.score = 0
        return observation
    
    def preprocess(self,observation):
        # Transformation de l'image RGB en nuance de gris => Entraînement plus rapide
        image_gris = cv2.cvtColor(observation,cv2.COLOR_RGB2GRAY)
        # Modifier la taille de l'image => Entraînement plus rapide
        image_retaillee = cv2.resize(image_gris,(self.taille_reduite,self.taille_reduite), interpolation = cv2.INTER_CUBIC)
        # Specificité pour stable_baselines
        image_retaillee_gris_finale = np.reshape(image_retaillee,(self.taille_reduite,self.taille_reduite,1))
        return image_retaillee_gris_finale
    
    def close(self):
        self.game.close()

In [4]:
from stable_baselines3 import PPO
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.vec_env import DummyVecEnv, VecFrameStack
import os

/home/alexandre/INSA/GM5/T2/PBIA/PBIA_Env/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
LOG_DIR = './logs/'
OPT_DIR = './opt/'

## Définition du CallBack

In [6]:
from stable_baselines3.common.callbacks import BaseCallback

In [7]:
class TrainAndLoggingCallback(BaseCallback):
    def __init__(self,check_freq,save_path, verbose=1):
        super(TrainAndLoggingCallback,self).__init__(verbose)
        self.check_freq = check_freq
        self.save_path = save_path

    def _init_callback(self):
        if self.save_path is not None:
            os.makedirs(self.save_path,exist_ok=True)

    def _on_step(self):
        if self.n_calls % self.check_freq == 0:
            model_path = os.path.join(self.save_path,'best_model_ppo_{}'.format(self.n_calls))
            self.model.save(model_path)

        return True

In [8]:
CHECKPOINT_DIR = './train/'

In [9]:
callback = TrainAndLoggingCallback(check_freq=50000, save_path = CHECKPOINT_DIR)

## Préparation de l'environnement

In [10]:
# Crée un environnement
env = StreetFighter()
# Permet d'extraire la recompense moyenne et la longueur moyenne d'un episode
env = Monitor(env,LOG_DIR)
#Nécessaire pour Stable Baselines
env = DummyVecEnv([lambda: env])
#Empile 4 images consécutives pour donner la perception de mouvement
env = VecFrameStack(env, 4, channels_order='last')

## Continuer Entrainement d'un modèle déjà sauvegardé

In [11]:
model = PPO.load(os.path.join(CHECKPOINT_DIR, 'sf2_last_save_ppo'), env=env)

Wrapping the env in a VecTransposeImage.


In [12]:
model.learn(total_timesteps = 5000000, callback = callback,reset_num_timesteps=False,tb_log_name = 'SF_PPO')

Logging to ./logs/SF_PPO_0
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1.67e+04 |
|    ep_rew_mean     | 7.69e+04 |
| time/              |          |
|    fps             | 116954   |
|    iterations      | 1        |
|    time_elapsed    | 21       |
|    total_timesteps | 2560320  |
---------------------------------
-------------------------------------------
| rollout/                |               |
|    ep_len_mean          | 1.67e+04      |
|    ep_rew_mean          | 7.69e+04      |
| time/                   |               |
|    fps                  | 68492         |
|    iterations           | 2             |
|    time_elapsed         | 37            |
|    total_timesteps      | 2562880       |
| train/                  |               |
|    approx_kl            | 0.00022094818 |
|    clip_fraction        | 0             |
|    clip_range           | 0.369         |
|    entropy_loss         | -4.7          |
|    explained_va

ValueError: Expected parameter logits (Tensor of shape (64, 12)) of distribution Bernoulli(logits: torch.Size([64, 12])) to satisfy the constraint Real(), but found invalid values:
tensor([[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan, nan, nan, 